In [1]:
# Top level run file
# Run the code from here

# Import the user defined functions

from func_mors_dfs import create_mors_df_dict, filter_bab
from func_read_data import read_data
from func_score_agg import score_agg
from process_mors import process_mors

# File path and names. Names are used as keys in the data_dict
path = '/Users/ellengoddard/Desktop/working_data/'
f_names = ['core_10_table', 'mors_table', 'pai_table_v2',
           'swemwebs_table', 'whooley_table']
ext = '.csv'
# List of NaN indicators used on read in of data
na_list = [999, '', 'Not applicable', 'NULL', 'Undefined', 'Unknown']

# Read in all datasets
data_dict = read_data(path, f_names, ext, na_list)

# Run the sub functions to aggregate the scores for each measure
# and invert where necessary so a higher score is always better
data_dict = score_agg(data_dict, f_names)

# Define MORS dataset 
mors_df = data_dict['mors_table']

# Remove Baby Steps data from MORS 
mors_df_filtered = filter_bab(mors_df)

# Separate MORS into dataframes based on assessment tool 
mors_df_dict = create_mors_df_dict(mors_df_filtered)

processed_mors_dict = process_mors(mors_df_dict)

# Combine the original data_dict with the mors_df_dict
combined_data_dict = {**data_dict, **processed_mors_dict}





In [2]:
combined_data_dict["COS - MORSC"].info()

<class 'pandas.core.frame.DataFrame'>
Index: 190 entries, 52 to 581
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   leap_user_key            190 non-null    int64         
 1   assessment_date          190 non-null    datetime64[ns]
 2   assessment_tool          190 non-null    category      
 3   event                    189 non-null    category      
 4   age_at_registration      186 non-null    float64       
 5   ethnicity_b              183 non-null    category      
 6   home_language            175 non-null    category      
 7   lone_parent              2 non-null      category      
 8   lsoacode                 189 non-null    category      
 9   ward_name                189 non-null    category      
 10  ward_code                189 non-null    category      
 11  borough                  189 non-null    category      
 12  lsoaname                 189 non-null   

In [4]:
combined_data_dict["PRS - MORS"]

,leap_user_key,assessment_date,assessment_tool,event,age_at_registration,ethnicity_b,home_language,lone_parent,lsoacode,ward_name,...,ptt_reached_dosage_yn,ptt_date_reached_dosage,cos_total_dosage,cos_reached_dosage_yn,cos_date_reached_dosage,warmth_total,invasion_total,invasion_final,warmth_final,total_final
0,504848,2018-11-01,PRS - MORS,PRS - 10th Session,30.0,NaN,English,Yes,NaN,NaN,...,None,NaT,NaN,None,None,32.0,8.0,27.0,32.0,59.0
1,504837,2019-10-01,PRS - MORS,PRS - 10th Session,26.0,White\r,German,No,NaN,NaN,...,None,NaT,NaN,None,None,30.0,8.0,27.0,30.0,57.0
2,504833,2019-07-01,PRS - MORS,PRS - 10th Session,22.0,Black\r,English,Yes,NaN,NaN,...,None,NaT,NaN,None,None,33.0,10.0,25.0,33.0,58.0
3,504745,2019-05-01,PRS - MORS,PRS - 10th Session,NaN,White\r,English,NaN,NaN,NaN,...,None,NaT,NaN,None,None,25.0,14.0,21.0,25.0,46.0
4,504721,2018-05-01,PRS - MORS,PRS - Initial,NaN,NaN,English,NaN,NaN,NaN,...,None,NaT,NaN,None,None,21.0,10.0,25.0,21.0,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,394473,2021-03-01,PRS - MORS,PRS - Initial,34.0,White\r,Other,NaN,E01033207,Oval,...,None,NaT,NaN,None,None,14.0,11.0,24.0,14.0,38.0
579,440737,2021-10-01,PRS - MORS,PRS - Initial,0.0,White\r,English,NaN,NaN,NaN,...,None,NaT,NaN,None,None,22.0,12.0,23.0,22.0,45.0
580,440737,2022-02-01,PRS - MORS,PRS - 10th Session,0.0,White\r,English,NaN,NaN,NaN,...,None,NaT,NaN,None,None,23.0,11.0,24.0,23.0,47.0
593,445635,2021-05-01,PRS - MORS,PRS - Initial,30.0,White\r,English,NaN,NaN,NaN,...,None,NaT,NaN,None,None,15.0,10.0,25.0,15.0,40.0


In [3]:
combined_data_dict["COS - MORSB"]["assessment_tool"]

74     COS - MORSB
109    COS - MORSB
112    COS - MORSB
122    COS - MORSB
128    COS - MORSB
138    COS - MORSB
144    COS - MORSB
146    COS - MORSB
149    COS - MORSB
150    COS - MORSB
162    COS - MORSB
164    COS - MORSB
197    COS - MORSB
198    COS - MORSB
252    COS - MORSB
269    COS - MORSB
270    COS - MORSB
271    COS - MORSB
272    COS - MORSB
340    COS - MORSB
346    COS - MORSB
347    COS - MORSB
348    COS - MORSB
352    COS - MORSB
370    COS - MORSB
375    COS - MORSB
420    COS - MORSB
421    COS - MORSB
423    COS - MORSB
530    COS - MORSB
532    COS - MORSB
576    COS - MORSB
583    COS - MORSB
594    COS - MORSB
Name: assessment_tool, dtype: category
Categories (1, object): ['COS - MORSB']